<a href="https://colab.research.google.com/github/rkb-1/mrca/blob/master/Kinematic_Analysis_of_RRP_robot_Spatial_Representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Kinematics of Open loop chains - Spatial Representation
-Rohit Kumar (r.kumar@dfki.de)

## Helper functions

In [20]:
# Import required libraries
import numpy as np
from sympy import Matrix, symbols, Identity, cos, sin, zeros, simplify, init_printing, print_latex

def SE3AdjInvMatrix(C):
    """Compute Inverse of (6x6) Adjoint Matrix for SE(3)

    Args:
        C ([type]): [description]

    Returns:
        sympy.Matrix: Inverse of (6x6) Adjoint Matrix
    """
    AdInv = Matrix([[C[0, 0], C[1, 0], C[2, 0], 0, 0, 0],
                    [C[0, 1], C[1, 1], C[2, 1], 0, 0, 0],
                    [C[0, 2], C[1, 2], C[2, 2], 0, 0, 0],
                    [-C[2, 3]*C[1, 0]+C[1, 3]*C[2, 0], C[2, 3]*C[0, 0]-C[0, 3]*C[2, 0],
                        (-C[1, 3])*C[0, 0]+C[0, 3]*C[1, 0], C[0, 0], C[1, 0], C[2, 0]],
                    [-C[2, 3]*C[1, 1]+C[1, 3]*C[2, 1], C[2, 3]*C[0, 1]-C[0, 3]*C[2, 1],
                        (-C[1, 3])*C[0, 1]+C[0, 3]*C[1, 1], C[0, 1], C[1, 1], C[2, 1]],
                    [-C[2, 3]*C[1, 2]+C[1, 3]*C[2, 2], C[2, 3]*C[0, 2]-C[0, 3]*C[2, 2],
                        (-C[1, 3])*C[0, 2]+C[0, 3]*C[1, 2], C[0, 2], C[1, 2], C[2, 2]]])
    return AdInv

def SE3AdjMatrix(C):
    """Compute (6x6) Adjoint Matrix for SE(3)

    Args:
        C ([type]): [description]

    Returns:
    sympy.Matrix: (6x6) Adjoint Matrix
    """
    Ad = Matrix([[C[0, 0], C[0, 1], C[0, 2], 0, 0, 0],
                 [C[1, 0], C[1, 1], C[1, 2], 0, 0, 0],
                 [C[2, 0], C[2, 1], C[2, 2], 0, 0, 0],
                 [-C[2, 3]*C[1, 0]+C[1, 3]*C[2, 0], -C[2, 3]*C[1, 1]+C[1, 3]*C[2, 1],
                  -C[2, 3]*C[1, 2]+C[1, 3]*C[2, 2], C[0, 0], C[0, 1], C[0, 2]],
                 [C[2, 3]*C[0, 0]-C[0, 3]*C[2, 0],  C[2, 3]*C[0, 1]-C[0, 3]*C[2, 1],
                     C[2, 3]*C[0, 2]-C[0, 3]*C[2, 2], C[1, 0], C[1, 1], C[1, 2]],
                 [-C[1, 3]*C[0, 0]+C[0, 3]*C[1, 0], -C[1, 3]*C[0, 1]+C[0, 3]*C[1, 1],
                  -C[1, 3]*C[0, 2]+C[0, 3]*C[1, 2], C[2, 0], C[2, 1], C[2, 2]]])
    return Ad

def SE3adMatrix(X):
    """Compute (6x6) adjoint Matrix for SE(3) 
        - also known as spatial cross product in the literature.

    Args:
        X ([type]): [description]

    Returns:
        sympy.Matrix: (6x6) adjoint Matrix
    """
    ad = Matrix([[0, -X[2, 0], X[1, 0], 0, 0, 0],
                 [X[2, 0], 0, -X[0, 0], 0, 0, 0],
                 [-X[1, 0], X[0, 0], 0, 0, 0, 0],
                 [0, -X[5, 0], X[4, 0], 0, -X[2, 0], X[1, 0]],
                 [X[5, 0], 0, -X[3, 0], X[2, 0], 0, -X[0, 0]],
                 [-X[4, 0], X[3, 0], 0, -X[1, 0], X[0, 0], 0]])
    return ad

def SE3Exp(XX, t):
    """compute exponential mapping for SE(3).

    Args:
        XX ([type]): [description]
        t ([type]): [description]

    Returns:
        [type]: [description]
    """
    X = XX.T
    xi = Matrix(X[0:3])
    eta = Matrix(X[3:6])
    xihat = Matrix([[0, -xi[2], xi[1]],
                    [xi[2], 0, -xi[0]],
                    [-xi[1], xi[0], 0]])
    R = Matrix(Identity(3)) + sin(t)*xihat + (1-cos(t))*(xihat*xihat)
    if xi == zeros(3, 1):
        p = eta * t
    else:
        p = Matrix(Identity(3)*t + (1-cos(t))*xihat + (t-sin(t))*(xihat*xihat))*eta
    C = R.row_join(p).col_join(Matrix([0, 0, 0, 1]).T)
    return C

def SE3Inv(C):
    """Compute analytical inverse of exponential mapping for SE(3).

    Args:
        C ([type]): [description]

    Returns:
        [type]: [description]
    """
    CInv = Matrix([[C[0, 0], C[1, 0], C[2, 0], -C[0, 0]*C[0, 3]-C[1, 0]*C[1, 3]-C[2, 0]*C[2, 3]],
                   [C[0, 1], C[1, 1], C[2, 1], -C[0, 1] *
                       C[0, 3]-C[1, 1]*C[1, 3]-C[2, 1]*C[2, 3]],
                   [C[0, 2], C[1, 2], C[2, 2], -C[0, 2] *
                       C[0, 3]-C[1, 2]*C[1, 3]-C[2, 2]*C[2, 3]],
                   [0, 0, 0, 1]])
    return CInv

def SO3Exp(x, t):
    """Compute exponential mapping for SO(3) (Rotation Matrix).

    Args:
        x (sympy.Matrix): Rotation axis
        t (double): Rotation angle

    Returns:
        sympy.Matrix: Rotation Matrix
    """
    xhat = Matrix([[0, -x[2, 0], x[1, 0]],
                   [x[2, 0], 0, -x[0, 0]],
                   [-x[1, 0], x[0, 0], 0]])
    R = Matrix(Identity(3)) + sin(t) * xhat + (1-cos(t))*(xhat*xhat)
    return R

def InertiaMatrix(Ixx, Ixy, Ixz, Iyy, Iyz, Izz):
    I = Matrix([[Ixx, Ixy, Ixz],
                [Ixy, Iyy, Iyz],
                [Ixz, Iyz, Izz]])
    return I

def TransformationMatrix(r=Matrix(Identity(3)), t=zeros(3, 1)):
    """Build Transformation matrix from rotation and translation

    Args:
        r (sympy.Matrix): SO(3) Rotation matrix (3,3). Defaults to sympy.Matrix(Identity(3))
        t (sympy.Matrix): Translation vector (3,1). Defaults to sympy.zeros(3,1)

    Returns:
        sympy.Matrix: Transformation matrix
    """
    T = r.row_join(t).col_join(Matrix([[0, 0, 0, 1]]))
    return T

def MassMatrixMixedData(m, Theta, COM):
    """Build mass-inertia matrix in SE(3) from mass, inertia and center of mass information.

    Args:
        m (float): Mass.
        Theta (sympy.Matrix): Inertia.
        COM (sympy.Matrix): Center of Mass.

    Returns:
        sympy.Matrix: Mass-inertia Matrix.
    """
    M = Matrix([[Theta[0, 0], Theta[0, 1], Theta[0, 2], 0, (-COM[2])*m, COM[1]*m],
                [Theta[0, 1], Theta[1, 1], Theta[1, 2],
                    COM[2]*m, 0, (-COM[0]*m)],
                [Theta[0, 2], Theta[1, 2], Theta[2, 2],
                    (-COM[1])*m, COM[0]*m, 0],  
                [0, COM[2]*m, (-COM[1]*m), m, 0, 0],
                [(-COM[2])*m, 0, COM[0]*m, 0, m, 0],
                [COM[1]*m, (-COM[0])*m, 0, 0, 0, m]])
    return M

# Kinematic Analysis of RRP robot

![picture](https://drive.google.com/uc?export=view&id=1JYMjAVi0WXi56IbYKLWqYV_ASSMInpe2)


## Spatial Representation of Screw axes


In [21]:
l = symbols("l", real=1)  # link lenghts

In [22]:
s1_hat = Matrix([1,0,0])
s1_o = Matrix([0,0,0])
display(s1_hat,s1_o)

Matrix([
[1],
[0],
[0]])

Matrix([
[0],
[0],
[0]])

In [23]:
s2_hat = Matrix([0,1,0])
s2_o = Matrix([0,0,0])
display(s2_hat,s2_o)

Matrix([
[0],
[1],
[0]])

Matrix([
[0],
[0],
[0]])

In [24]:
s3_hat = Matrix([0,0,1])
display(s3_hat)

Matrix([
[0],
[0],
[1]])

In [25]:
S1 = Matrix([s1_hat, s1_o.cross(s1_hat)])
S2 = Matrix([s2_hat, s2_o.cross(s2_hat)])
S3 = Matrix([Matrix([0,0,0]), s3_hat])
print("S1:")
display(S1)
print("S2:")
display(S2)
print("S3:")
display(S3)
S = S1.row_join(S2).row_join(S3)
print("Screw axes (arranged column wise for the joints) :\n")
display(S)

S1:


Matrix([
[1],
[0],
[0],
[0],
[0],
[0]])

S2:


Matrix([
[0],
[1],
[0],
[0],
[0],
[0]])

S3:


Matrix([
[0],
[0],
[0],
[0],
[0],
[1]])

Screw axes (arranged column wise for the joints) :



Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 0],
[0, 0, 0],
[0, 0, 0],
[0, 0, 1]])

## Reference configuration of the bodies (body fixed representation) at zero position
A general configuration is given by a rotation matrix $\mathbf{R}$ and position vector $\mathbf{p}$ as follows.
\begin{equation}
        \mathbf{T} = \begin{bmatrix}
                \mathbf{R} & \textbf{p} \\ \mathbf{0} & 1
            \end{bmatrix}
    \end{equation}



In [26]:
p0e = Matrix([0,0,l])
R0e = Matrix(Identity(3))

T0e = TransformationMatrix(R0e,p0e)

print("Zero configuration of end-effector pose: \n")
display(T0e)

Zero configuration of end-effector pose: 



Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, l],
[0, 0, 0, 1]])

## Position Analysis

### The product of exponential (POE) formula is given by
\begin{equation}
  {}^0\mathbf T_n(\mathbf q) = \exp([\mathbf S_1]q_1) \exp([\mathbf S_2]q_2) \ldots \exp([\mathbf S_n]q_n) {}^0\mathbf T_n(\mathbf 0) 
  \label{eqn_POE}
 \end{equation}


### Define the generalised coordinates for the system, $q_1$ $q_2$ $q_3$

In [27]:
q1, q2, q3 = symbols("q1 q2 q3", real=1)  # joint positions
q = Matrix([q1,q2,q3])

T0q = simplify(SE3Exp(S1,q1) * SE3Exp(S2,q2) * SE3Exp(S3,q3) * T0e)
print("Forward kinematics : \n")
display(T0q)

Forward kinematics : 



Matrix([
[         cos(q2),       0,          sin(q2),          (l + q3)*sin(q2)],
[ sin(q1)*sin(q2), cos(q1), -sin(q1)*cos(q2), -(l + q3)*sin(q1)*cos(q2)],
[-sin(q2)*cos(q1), sin(q1),  cos(q1)*cos(q2),  (l + q3)*cos(q1)*cos(q2)],
[               0,       0,                0,                         1]])

## Velocity Analysis

The spatial twist $\mathbf V$ is given by $[\mathbf V] = \dot{\mathbf T} \mathbf T^{-1} \in se(3)$

\begin{equation*}
             \mathbf V = \underbrace{\mathbf{Ad}_{\exp([\mathbf S_1]q_1)}(\mathbf{S_1})}_{\mathbf J_{1}} \dot{q}_1
                        + \underbrace{\mathbf{Ad}_{\exp([\mathbf S_1]q_1)
                                \exp([\mathbf S_2]q_2)}(\mathbf{S_2})}_{\mathbf J_{2}} \dot{q}_2 + \ldots
\end{equation*}

where $\mathbf J_{i} = \mathbf{Ad}_{\mathbf T_i}(\mathbf S_i)$ 
with  $\mathbf T_{i} = \exp([\mathbf S_1]q_1) \dots \exp([\mathbf S_{i}]q_{i})$ is the instantaneous screw coordinate vector of
        the $i^\text{th}$ joint


In [28]:
print("Spatial Jacobian at zero position is same as screw axis")
display(S)
Js_1 = SE3AdjMatrix(SE3Exp(S1,q1)) * S1
Js_2 = SE3AdjMatrix(SE3Exp(S1,q1) * SE3Exp(S2,q2)) * S2
Js_3 = SE3AdjMatrix(SE3Exp(S1,q1) * SE3Exp(S2,q2) * SE3Exp(S3,q3)) * S3
Js = simplify(Js_1.row_join(Js_2).row_join(Js_3))
print("Spatial Jacobian: ")
display(Js)

Spatial Jacobian at zero position is same as screw axis


Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 0],
[0, 0, 0],
[0, 0, 0],
[0, 0, 1]])

Spatial Jacobian: 


Matrix([
[1,       0,                0],
[0, cos(q1),                0],
[0, sin(q1),                0],
[0,       0,          sin(q2)],
[0,       0, -sin(q1)*cos(q2)],
[0,       0,  cos(q1)*cos(q2)]])

#### Recursive formulation of Spatial velocity
The recursive formulation of the velocity of any body $i$ can be expressed as a sum of the velocity of previous body $i-1$ and velocity across the joint $\dot{q}_i$.

\begin{equation*}
 \mathbf V_i = \mathbf V_{i-1} + \mathbf{Ad}_{\mathbf T_{i}}(\mathbf S_i) \dot{q}_i
\end{equation*}


In [29]:
q1d, q2d, q3d = symbols("qdot1 qdot2 qdot3", real=1)  # joint positions
qd = Matrix([q1d,q2d,q3d])

Vs1 = Js_1*q1d
Vs2 = Vs1 + Js_2*q2d
Vs3 = Vs2 + Js_3*q3d

print("Spatial Velocities: \n")
display(simplify(Vs1.row_join(Vs2).row_join(Vs3)))
Vs = Js*qd
print("Spatial twist of end-effector: ")
display(Vs)

Spatial Velocities: 



Matrix([
[qdot1,         qdot1,                  qdot1],
[    0, qdot2*cos(q1),          qdot2*cos(q1)],
[    0, qdot2*sin(q1),          qdot2*sin(q1)],
[    0,             0,          qdot3*sin(q2)],
[    0,             0, -qdot3*sin(q1)*cos(q2)],
[    0,             0,  qdot3*cos(q1)*cos(q2)]])

Spatial twist of end-effector: 


Matrix([
[                 qdot1],
[         qdot2*cos(q1)],
[         qdot2*sin(q1)],
[         qdot3*sin(q2)],
[-qdot3*sin(q1)*cos(q2)],
[ qdot3*cos(q1)*cos(q2)]])

## Acceleration Analysis
Differentiating velocity equation with respect to time,
\begin{equation*}
\begin{aligned}
 \dot{\mathbf V} = \underbrace{\mathbf{Ad}_{\exp([\mathbf S_1]q_1)}(\mathbf{S_1})}_{\mathbf J_{1}}
 \ddot{q}_1 + \underbrace{\frac{d}{dt}\mathbf{Ad}_{\exp([\mathbf
 S_1]q_1)}(\mathbf{S_1})}_{\frac{d}{dt}\mathbf J_{1}} \dot{q}_1 + \\
 \underbrace{\mathbf{Ad}_{\exp([\mathbf S_1]q_1) \exp([\mathbf S_2]q_2)}(\mathbf{S_2})}_{\mathbf
 J_{2}} \ddot{q}_2 + \underbrace{\frac{d}{dt}\mathbf{Ad}_{\exp([\mathbf S_1]q_1) \exp([\mathbf
S_2]q_2)}(\mathbf{S_2})}_{\frac{d}{dt}\mathbf J_{2}} \dot{q}_2 + \ldots \\
\end{aligned}
\end{equation*}

\begin{equation*}
      \dot{\mathbf V} = \mathbf J \ddot{\mathbf q} + \dot{\mathbf J} \dot{\mathbf q}
\end{equation*}
 
 
Given two twists $\mathbf V_1 = (\mathbf{\omega}_1, \mathbf{v}_1)$ and 
$\mathbf V_2 = (\mathbf{\omega}_2, \mathbf{v}_2)$, the Lie Bracket 
$\left[ \mathbf V_1, \mathbf V_2 \right]$ of $\mathbf V_1$ and $\mathbf V_2$
written either as $\left[\mathbf{ad}_{\mathbf V_1}\right] \mathbf V_2$ in matrix form or $\mathbf{ad}_{\mathbf V_1}(\mathbf V_2)$ in the form of a mapping, is defined as

 \begin{equation*}
  \left[
 \begin{matrix}
 \left[\mathbf{\omega}_1\right] & \mathbf 0 \\
 \left[\mathbf{v}_1\right] & \left[\mathbf{\omega}_1\right] \\
 \end{matrix}
\right]
  \left[
 \begin{matrix}
\mathbf{\omega}_2	\\
 \mathbf{v}_2	\\
 \end{matrix}
\right]
= \left[\mathbf{ad}_{\mathbf V_1}\right] \mathbf V_2
= \mathbf{ad}_{\mathbf V_1}(\mathbf V_2)
\in \mathbb{R}^{6 \times 6}
 \end{equation*}
\begin{equation*}
        \dot{\mathbf J}_i = [\mathbf V_i, \mathbf J_i]
\end{equation*}


In [30]:
q1dd, q2dd, q3dd = symbols("qddot1 qddot2 qddot3", real=1)  # joint positions
qdd = Matrix([q1dd,q2dd,q3dd])

Jsdot1 = zeros(6,1)
Jsdot2 = SE3adMatrix(Vs1)*Js_2
Jsdot3 = SE3adMatrix(Vs2)*Js_3

Jsdot = simplify(Jsdot1.row_join(Jsdot2).row_join(Jsdot3))
print("Spatial Jacobian derivative (Js_dot)")
display(Jsdot)


Spatial Jacobian derivative (Js_dot)


Matrix([
[0,              0,                                              0],
[0, -qdot1*sin(q1),                                              0],
[0,  qdot1*cos(q1),                                              0],
[0,              0,                                  qdot2*cos(q2)],
[0,              0, -qdot1*cos(q1)*cos(q2) + qdot2*sin(q1)*sin(q2)],
[0,              0, -qdot1*sin(q1)*cos(q2) - qdot2*sin(q2)*cos(q1)]])

The acceleration of any body $i$ in the kinematic chain can be expressed in the following
summand form:

\begin{equation*}
 \dot{\mathbf V}_i = \sum_{j \leq i} \left( \mathbf J_j \ddot{q}_j + [\mathbf V_j, \mathbf J_j]
 \dot{q}_j \right)
\end{equation*}

which reveals the recursive nature of the acceleration computation i.e. the
acceleration of any body $i$ can be expressed as a sum of the acceleration of previous body
$i-1$ and acceleration across the joint $\ddot{q}_i$.

\begin{equation*}
  \dot{\mathbf V}_i =  \dot{\mathbf V}_{i-1} + \mathbf J_i \ddot{q}_i +[\mathbf V_{i},\mathbf J_i] \dot{q}_i
\end{equation*}

This can be further simplified:

\begin{equation*}
\begin{aligned}
  \dot{\mathbf V}_i =  \dot{\mathbf V}_{i-1} + \mathbf J_i \ddot{q}_i + [\mathbf V_{i-1},\mathbf V_i] \\
  \text{or} \quad \dot{\mathbf V}_i =  \dot{\mathbf V}_{i-1} + \mathbf{Ad}_{\mathbf T_i}(\mathbf
  S_i) \ddot{q}_i + \mathbf{ad}_{\mathbf V_{i-1}}(\mathbf V_i)
\end{aligned}
\end{equation*}


In [31]:
Vsdot1 = Js_1*q1dd 
Vsdot2 = Vsdot1 + Js_2*q2dd + SE3adMatrix(Vs1)*Vs2
Vsdot3 = Vsdot2 + Js_3*q3dd + SE3adMatrix(Vs2)*Vs3

print("Spatial Accelerations: ")
display(simplify(Vsdot1.row_join(Vsdot2).row_join(Vsdot3)))
Vsdot = Js*qdd + Jsdot*qd
print("Spatial acceleration of end-effector: ")
display(Vsdot)

Spatial Accelerations: 


Matrix([
[qddot1,                               qddot1,                                                                              qddot1],
[     0, qddot2*cos(q1) - qdot1*qdot2*sin(q1),                                                qddot2*cos(q1) - qdot1*qdot2*sin(q1)],
[     0, qddot2*sin(q1) + qdot1*qdot2*cos(q1),                                                qddot2*sin(q1) + qdot1*qdot2*cos(q1)],
[     0,                                    0,                                                qddot3*sin(q2) + qdot2*qdot3*cos(q2)],
[     0,                                    0, -qddot3*sin(q1)*cos(q2) - qdot1*qdot3*cos(q1)*cos(q2) + qdot2*qdot3*sin(q1)*sin(q2)],
[     0,                                    0,  qddot3*cos(q1)*cos(q2) - qdot1*qdot3*sin(q1)*cos(q2) - qdot2*qdot3*sin(q2)*cos(q1)]])

Spatial acceleration of end-effector: 


Matrix([
[                                                                          qddot1],
[                                            qddot2*cos(q1) - qdot1*qdot2*sin(q1)],
[                                            qddot2*sin(q1) + qdot1*qdot2*cos(q1)],
[                                            qddot3*sin(q2) + qdot2*qdot3*cos(q2)],
[-qddot3*sin(q1)*cos(q2) + qdot3*(-qdot1*cos(q1)*cos(q2) + qdot2*sin(q1)*sin(q2))],
[ qddot3*cos(q1)*cos(q2) + qdot3*(-qdot1*sin(q1)*cos(q2) - qdot2*sin(q2)*cos(q1))]])

## Relationship with body-fixed representation

We can establish a relationship between spatial and body twist. First we
        note:

\begin{equation*}
    [\mathbf{V}_{b}] = \mathbf{T}^{-1}\dot{\mathbf{T}} = \mathbf{T}^{-1}[\mathbf{V}_{s}]\mathbf{T}.
\end{equation*}

And furthermore:

\begin{align*}
    [\mathbf{V}_{s}] &= \dot{\mathbf{T}}\mathbf{T}^{-1} = \mathbf{T}[\mathbf{V}_{b}]\mathbf{T}^{-1} \\
                   &= \begin{bmatrix}
        \mathbf{R}[\mathbf{\omega}_{b}]\mathbf{R}^{T} &
        -\mathbf{R}[\mathbf{\omega}_{b}]\mathbf{R}^{T}\mathbf{p}+\mathbf{R}\mathbf{v}_{b} \\
        0 & 0
    \end{bmatrix}.
\end{align*}

We can use this to formulate the following useful relation:

\begin{equation*}
    \begin{bmatrix}
        \mathbf{\omega}_{s} \\ \mathbf{v}_{s}
    \end{bmatrix} =
    \begin{bmatrix}
        \mathbf{R} & 0 \\
        [\mathbf{p}]\mathbf{R} & \mathbf{R}
    \end{bmatrix}
    \begin{bmatrix}
        \mathbf{\omega}_{b} \\ \mathbf{v}_{b}
    \end{bmatrix}
\end{equation*}

Conceptually this is very useful to change  the reference frame of (not only) twists. For a
        $\mathbf{T}=(\mathbf{R}, \mathbf{p})\in SE(3)$, we call

\begin{equation*}
    [\mathbf{Ad}_{\mathbf{T}}] := \begin{bmatrix}
        \mathbf{R} & 0 \\
        [\mathbf{p}]\mathbf{R} & \mathbf{R}
    \end{bmatrix} \in \mathbb{R}^{6\times 6}
\end{equation*}


In [32]:
Jb = simplify(SE3AdjInvMatrix(T0q)*Js)
print("Body Jacobian: \n")
display(Jb)

Body Jacobian: 



Matrix([
[          cos(q2),      0, 0],
[                0,      1, 0],
[          sin(q2),      0, 0],
[                0, l + q3, 0],
[-(l + q3)*cos(q2),      0, 0],
[                0,      0, 1]])

In [33]:
Vbdot= SE3AdjInvMatrix(T0q)*Vsdot
print("Body acceleration of end-effector")
display(simplify(Vbdot))

Body acceleration of end-effector


Matrix([
[                                                                        qddot1*cos(q2) - qdot1*qdot2*sin(q2)],
[                                                                                                      qddot2],
[                                                                        qddot1*sin(q2) + qdot1*qdot2*cos(q2)],
[                                                                          l*qddot2 + q3*qddot2 + qdot2*qdot3],
[-l*qddot1*cos(q2) + l*qdot1*qdot2*sin(q2) - q3*qddot1*cos(q2) + q3*qdot1*qdot2*sin(q2) - qdot1*qdot3*cos(q2)],
[                                                                                                      qddot3]])